In [1]:
%cd ../..


/home/nazif/thesis/mirscribe


In [2]:
import pandas as pd 
import os
import gc

from scripts.utils import *
from scripts.features import generate_is_mirna_column
from scripts.ensembl import generate_transcript_id_and_gene_name_columns

In [3]:
def read_vcf_file(file_name):
    return pd.read_csv(f"data/sample_vcfs/{file_name}.vcf", sep="\t", header=None, 
                       names=["chr", "pos", "id", "ref", "alt"])

def create_id_column(df):
    df['id'] = df["id"] + df['chr'] + '_' + df['pos'].astype(str) + '_' + df['ref'] + '_' + df['alt']
    return df

def calculate_allele_lengths(df):
    df["ref_len"] = df["ref"].str.len()
    df["alt_len"] = df["alt"].str.len()
    return df

def fetch_nucleotides(df):
    df['fetched_nucleotides'] = ''
    df.loc[df['ref_len'] > 1, 'fetched_nucleotides'] = df.apply(
        lambda x: get_nucleotides_in_interval(x['chr'], x['pos'], x["pos"]+x["ref_len"]-1), axis=1)
    df.loc[df['ref_len'] == 1, 'fetched_nucleotides'] = df.apply(
        lambda x: get_nucleotide_at_position(x['chr'], x['pos']), axis=1)
    return df

def check_nucleotides_match(df):
    df['is_nucleotides_same'] = df["fetched_nucleotides"] == df["ref"]
    return df

def calculate_ref_positions(df):
    df["ref_start"] = df["pos"]
    df["ref_end"] = df["pos"] + df["ref_len"]
    return df

def fetch_flanking_sequences(df):
    df['upstream_sequence'] = df.apply(
        lambda x: get_nucleotides_in_interval(x['chr'], x['ref_start']-30, x["ref_start"]-1), axis=1)
    df['downstream_sequence'] = df.apply(
        lambda x: get_nucleotides_in_interval(x['chr'], x['ref_end'], x["ref_end"]+29), axis=1)
    return df

def construct_sequences(df):
    df['sequence'] = df["upstream_sequence"] + df["ref"] + df["downstream_sequence"]
    df['mutated_sequence'] = df["upstream_sequence"] + df["alt"] + df["downstream_sequence"]
    return df

def generate_columns(df, grch):
    df = generate_is_mirna_column(df, grch=grch)
    df = generate_transcript_id_and_gene_name_columns(df, grch=grch)
    return df

def split_data(df):
    case_1 = df[df.is_mirna == 0]
    case_2 = df[df.is_mirna == 1]
    return case_1, case_2

def save_data(case_1, case_2, file_name, file_path):
    relevant_cols = ["id", "sequence", "mutated_sequence"]
    case_1 = case_1[relevant_cols]
    case_2 = case_2[relevant_cols]

    case_1.to_csv(f"{file_path}/{file_name}_case1.csv", index=False)

    if len(case_2) > 0:
        print(f"{len(case_2)} case 2 mutations were found on {file_name}.")
        case_2.to_csv(f"{file_path}/{file_name}_case2.csv", index=False)
    else:
        print(f"No case 2 mutations were found on {file_name}.")

def pipeline(file_name):
    df = read_vcf_file(file_name)
    df = create_id_column(df)
    df = calculate_allele_lengths(df)
    df = fetch_nucleotides(df)
    df = check_nucleotides_match(df)
    df = calculate_ref_positions(df)
    df = fetch_flanking_sequences(df)
    df = construct_sequences(df)
    df = generate_columns(df, grch=37)
    case_1, case_2 = split_data(df)
    save_data(case_1, case_2, file_name, "results/vcf_results")
    
    del df, case_1, case_2
    gc.collect()


In [4]:
folder_path = "data/sample_vcfs"

# Get a list of all CSV files in the folder
csv_files = [os.path.splitext(file)[0] for file in os.listdir(folder_path) if file.endswith(".vcf")]

for file in csv_files:
    pipeline(file)

INFO:pyensembl.sequence_data:Loaded sequence dictionary from /home/nazif/thesis/data/pyensembl/GRCh37/ensembl75/Homo_sapiens.GRCh37.75.cdna.all.fa.gz.pickle
INFO:pyensembl.sequence_data:Loaded sequence dictionary from /home/nazif/thesis/data/pyensembl/GRCh37/ensembl75/Homo_sapiens.GRCh37.75.ncrna.fa.gz.pickle
INFO:pyensembl.sequence_data:Loaded sequence dictionary from /home/nazif/thesis/data/pyensembl/GRCh37/ensembl75/Homo_sapiens.GRCh37.75.pep.all.fa.gz.pickle


No case 2 mutations were found on PD4088a.


INFO:pyensembl.sequence_data:Loaded sequence dictionary from /home/nazif/thesis/data/pyensembl/GRCh37/ensembl75/Homo_sapiens.GRCh37.75.cdna.all.fa.gz.pickle
INFO:pyensembl.sequence_data:Loaded sequence dictionary from /home/nazif/thesis/data/pyensembl/GRCh37/ensembl75/Homo_sapiens.GRCh37.75.ncrna.fa.gz.pickle
INFO:pyensembl.sequence_data:Loaded sequence dictionary from /home/nazif/thesis/data/pyensembl/GRCh37/ensembl75/Homo_sapiens.GRCh37.75.pep.all.fa.gz.pickle


No case 2 mutations were found on PD3904a.


INFO:pyensembl.sequence_data:Loaded sequence dictionary from /home/nazif/thesis/data/pyensembl/GRCh37/ensembl75/Homo_sapiens.GRCh37.75.cdna.all.fa.gz.pickle
INFO:pyensembl.sequence_data:Loaded sequence dictionary from /home/nazif/thesis/data/pyensembl/GRCh37/ensembl75/Homo_sapiens.GRCh37.75.ncrna.fa.gz.pickle
INFO:pyensembl.sequence_data:Loaded sequence dictionary from /home/nazif/thesis/data/pyensembl/GRCh37/ensembl75/Homo_sapiens.GRCh37.75.pep.all.fa.gz.pickle


...

No case 2 mutations were found on PD4005a.


INFO:pyensembl.sequence_data:Loaded sequence dictionary from /home/nazif/thesis/data/pyensembl/GRCh37/ensembl75/Homo_sapiens.GRCh37.75.cdna.all.fa.gz.pickle
INFO:pyensembl.sequence_data:Loaded sequence dictionary from /home/nazif/thesis/data/pyensembl/GRCh37/ensembl75/Homo_sapiens.GRCh37.75.ncrna.fa.gz.pickle
INFO:pyensembl.sequence_data:Loaded sequence dictionary from /home/nazif/thesis/data/pyensembl/GRCh37/ensembl75/Homo_sapiens.GRCh37.75.pep.all.fa.gz.pickle


No case 2 mutations were found on PD4086a.


INFO:pyensembl.sequence_data:Loaded sequence dictionary from /home/nazif/thesis/data/pyensembl/GRCh37/ensembl75/Homo_sapiens.GRCh37.75.cdna.all.fa.gz.pickle
INFO:pyensembl.sequence_data:Loaded sequence dictionary from /home/nazif/thesis/data/pyensembl/GRCh37/ensembl75/Homo_sapiens.GRCh37.75.ncrna.fa.gz.pickle
INFO:pyensembl.sequence_data:Loaded sequence dictionary from /home/nazif/thesis/data/pyensembl/GRCh37/ensembl75/Homo_sapiens.GRCh37.75.pep.all.fa.gz.pickle


No case 2 mutations were found on PD4248a.
